In [1]:
import pandas as pd
import numpy as np
from scipy import stats # 求概率密度

In [2]:
dic1 = {'货运总量':[160,260,210,265,240,220,275,160,275,250],
        '工业总产值':[70,75,65,74,72,68,78,66,70,85],
        '农业总产值':[35,40,40,42,38,45,42,36,44,42],
        '居民非商业支出':[1.0,2.4,2.0,3.0,1.2,1.5,4.0,2.0,3.2,3.0]}
df1 = pd.DataFrame(dic1)
df1.index.name = '编号' 
# df1.insert(1,'常数项',[1 for i in range(len(df1))])
df1

,货运总量,工业总产值,农业总产值,居民非商业支出
编号,,,,
0,160,70,35,1.0
1,260,75,40,2.4
2,210,65,40,2.0
3,265,74,42,3.0
4,240,72,38,1.2
5,220,68,45,1.5
6,275,78,42,4.0
7,160,66,36,2.0
8,275,70,44,3.2


In [224]:
class Regreession():
    '''
    df: 第一列为Y，其他列为X变量的DataFrame
    a: 置信系数
    
    '''
    def __init__(self, df, a=0.05):
        self.df = df
        self.Y = df.iloc[:,0]
        X = df.iloc[:,1:]
        X.insert(0,'常数项',[1 for i in range(len(df))]) # X矩阵插入常数项
        self.X = X
        self.a = 0.05
        self.beta = Regreession.beta(self) # 调用下面的函数计算β
        self.y_hat = Regreession.y_hat(self)
        self.sigma2 = Regreession.sigma2(self)
        
    # 计算相关矩阵函数
    def r(self): 
    # 默认第一列为y
        r = np.zeros(shape=(self.df.shape[1],self.df.shape[1]))
        for n1,i in enumerate(self.df):
            for n2,j in enumerate(self.df):
                a = np.sum((self.df[i]-np.mean(self.df[i]))*(self.df[j]-np.mean(self.df[j]))) # 分子
                b = np.sqrt(np.sum(np.square(self.df[i]-np.mean(self.df[i])))*np.sum(np.square(self.df[j]-np.mean(self.df[j])))) # 分母
                r[n1,n2] = a/b

        r = np.around(r,2) # 保留2位小数
        print('简单相关系数矩阵：r='); print(r)
    
    # 计算β
    def beta(self):
        beta = np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(self.X),self.X)),np.transpose(self.X)),self.Y) # (X'X)^(-1)=np.linalg.inv(np.dot(np.transpose(X),X))
        beta_ = np.around(beta,4)
        output = f'回归方程：y = {beta_[0]}'
        output2 = [f' + {beta_[i]}·x_{i}i' for i in range(1,len(beta_))]
        print(output,''.join(output2))
        return beta
    
    # 预测y值
    def y_hat(self,X=False):
        if X == False:  # 加入指定x1，x2，x3，求一个y时
            y_hat = np.dot(self.X,self.beta)
        else:
            y_hat = np.dot(X,self.beta)
        return y_hat
    
    # 计算可决系数R2
    def R2(self):
        R2 = np.sum(np.square(self.y_hat-np.mean(self.Y)))/np.sum(np.square(self.Y-np.mean(self.Y)))
        print(f'可决系数 R^2 ：{R2}')
        # return R2

    # F检验
    def F_test(self): 
        F = ( np.sum(np.square(self.y_hat-np.mean(self.Y)))/(self.X.shape[1]-1) )/( np.sum(np.square(self.Y-self.y_hat))/(self.Y.shape[0]-self.X.shape[1]-1) )
        n1 = self.X.shape[1]-1; n2 = self.Y.shape[0]-self.X.shape[1]-1
        p_value = 1 - stats.f.cdf(F,n1,n2) # 累计分布求分位数
        print(f'F检验：P值 = {p_value}')

    # 计算^σ2
    def sigma2(self):
        e2 = np.sum(np.square(self.Y-self.y_hat))        # 残差平方和e^2
        sigma2_ = (1/(self.Y.shape[0]-self.X.shape[1]))*e2         # 方差预测^σ^2
        return sigma2_
    
    # t检验
    def t_test(self):
        '''
        confidence_interval: 是否显示置信水平
        a: 置信水平
        '''
        df = pd.DataFrame(index=self.X.columns,columns=['t检验_p值','置信区间'])
        df.index.name = '变量系数'
        X_X = np.linalg.inv(np.dot(np.transpose(self.X),self.X)) # (X'X)^(-1)
        for n,j in enumerate(self.beta):
            cjj = X_X[n,n]
            t_j = self.beta[n]/np.sqrt(self.sigma2*cjj)
            free_n = self.Y.shape[0]-self.X.shape[1] # 自由度
            p_j = 1 - stats.t.cdf(t_j,free_n)
            df.iloc[n,0] = p_j

            # 置信区间
            beta_lp = self.beta[n] + stats.t.ppf(self.a/2,free_n)
            beta_ub = self.beta[n] - stats.t.ppf(self.a/2,free_n)
            df.iloc[n,1] = f'[{beta_lp}，{beta_ub}]'

        return df
    
    # 预测y置信区间
    def y_hat_interval(self,X):
        X.insert(0,1) # 常数项
        y_hat = Regreession.y_hat(self,X)
        y_hat_lb = y_hat - 2*np.sqrt(self.sigma2)
        y_hat_ub = y_hat + 2*np.sqrt(self.sigma2)
        print(f'y的预测值为{y_hat}')
        print(f'y的95%预测区间为 [ {y_hat_lb}, {y_hat_ub} ]')
    
    # 总结果
    #def summary(self):
        

In [225]:
rs = Regreession(df1)

回归方程：y = -204.8273  + 1.9462·x_1i + 6.5256·x_2i + 13.7258·x_3i


In [226]:
# 简单相关系数矩阵 r
rs.r()

简单相关系数矩阵：r=
[[1.   0.57 0.73 0.72]
 [0.57 1.   0.26 0.55]
 [0.73 0.26 1.   0.55]
 [0.72 0.55 0.55 1.  ]]


In [227]:
# 可决系数 R^2
rs.R2()

可决系数 R^2 ：0.7339285745043419


In [228]:
# F检验
rs.F_test()

F检验：P值 = 0.06691053834051774


In [229]:
# t检验
rs.t_test()

,t检验_p值,置信区间
变量系数,,
常数项,0.857962,[-207.27418267840523，-202.38035898082188]
工业总产值,0.163166,[-0.500696269788643，4.3931274277947185]
农业总产值,0.0493461,[4.078701044848414，8.972524742431775]
居民非商业支出,0.167186,[11.27889225619615，16.172715953779512]


In [230]:
# y预测区间
rs.y_hat_interval(X=[75,42,3.1])

y的预测值为257.76463185396057
y的95%预测区间为 [ 202.92805135812614, 312.601212349795 ]
